In [ ]:
import yt

ds = yt.load("fm_mad/100Ms/torus.mhd_w_bcc.00050.athdf")
ds.print_stats()

slc = yt.SlicePlot(ds,  "x", ('athena_pp','velz'))
slc.set_cmap(('athena_pp','velz'), 'RdBu')
slc.show()


In [ ]:
import yt

ds = yt.load("fm_mad/100Ms/torus.mhd_w_bcc.00100.athdf")
ds.print_stats()

slc = yt.SlicePlot(ds,  "x", ('gas','density_gradient_magnitude'))
slc.show()
    

In [ ]:
ds.derived_field_list


# SCALE HEIGHT

In [ ]:
import yt
import numpy as np
from yt import derived_field
import yt
from PIL import Image

# Load your AthenaK dataset (replace filename as appropriate)
ds = yt.load("fm_mad/100Ms/torus.mhd_w_bcc.00100.athdf")

# Check if gradient fields are available
print(ds.derived_field_list)  # you should see ('gas', 'density_gradient_x'), etc.

# Define the derived field
@derived_field(name=("gas", "density_scale_height"),
                  units="code_length",  # or "code_length" if in code units
                  sampling_type="cell")
def _density_scale_height(field, data):
    rho = data["athena_pp", "dens"]
    grad = data["gas", "density_gradient_z"]
    H = rho / grad
    return H

# Now you can access this field just like any other
ad = ds.all_data()


In [ ]:
# Slice plot in the midplane
slc = yt.SlicePlot(ds, "x", ("gas", "density_scale_height"))
slc.set_log(("gas", "density_scale_height"), False)
slc.set_cmap(("gas", "density_scale_height"), "viridis")
slc.annotate_title("Density Scale Height (H = ρ / |∇ρ|)")
slc.save("scale_height_slice.png")
slc.show()


In [ ]:
# Create a 1D radial profile
prof = yt.create_profile(
    ad,
    "cylindrical_radius",
    ("gas", "density_scale_height"),
    weight_field=("gas", "cell_volume")
)

import matplotlib.pyplot as plt

plt.figure(figsize=(7,5))
plt.plot(prof.x.value, prof["gas", "density_scale_height"].value)
plt.xscale("log")
plt.yscale("log")
plt.xlabel("Radius [cm]")
plt.ylabel("Scale Height [cm]")
plt.title("Radial Density Scale Height Profile")
plt.grid(True, which="both")
plt.show()


In [ ]:
from yt import derived_field
import yt
from PIL import Image
@derived_field(name="mom_x",units="code_mass/code_length**3", sampling_type="cell", display_name="Momentum X",force_override=True)
def _mom_x(field, data):
    return data["athena_pp", "dens"] * data["athena_pp", "velx"]    
@derived_field(name="mom_y",units="code_mass/code_length**3", sampling_type="cell", display_name="Momentum Y",force_override=True)
def _mom_y(field, data):
    return data["athena_pp", "dens"] * data["athena_pp", "vely"]
@derived_field(name="mom_z", units="code_mass/code_length**3", sampling_type="cell", display_name="Momentum Z",force_override=True)
def _mom_z(field, data):
    return data["athena_pp", "dens"] * data["athena_pp", "velz"]
@derived_field(name="velocity_x",units="", sampling_type="cell", display_name="velocity_x",force_override=True)
def _velocity_x(field, data):
    return data["athena_pp", "velx"]
@derived_field(name="velocity_y",units="", sampling_type="cell", display_name="velocity_y",force_override=True)
def _velocity_y(field, data):
    return data["athena_pp", "vely"]
@derived_field(name="velocity_z",units="", sampling_type="cell", display_name="velocity_z",force_override=True)
def _velocity_z(field, data):
    return data["athena_pp", "velz"]
@derived_field(name="dens",units="code_mass/code_length**3", sampling_type="cell", display_name="dens",force_override=True)
def _dens(field, data):
    return data["athena_pp", "dens"]

# Define the derived field


import matplotlib.pyplot as plt

times = [0, 50, 75, 100]
images = []
for t in times:
    ds = yt.load(f"fm_mad/100Ms/torus.mhd_w_bcc.00{t:03d}.athdf")
    # make a slice plot of density using the disk as the data_source

    p = yt.SlicePlot(ds, "x", ('gas', 'dens'))
    p.zoom(32)
    try:
        p.annotate_velocity()
        p.annotate_contours(('athena_pp', 'bcc'), ncont=10, clim=(0.1, 10), colors='white')
    except Exception:
        # annotate_velocity can fail if velocities aren't available/compatible for this data_source
        pass
    # save the plot and load the saved image for composing in a matplotlib figure
    saved = p.save()  # returns a list of saved file paths
    images.append(Image.open(saved[0]))

# plot all four saved images in a 2x2 layout
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()
for ax, img, t in zip(axes, images, times):
    ax.imshow(img)
    ax.set_title(f"time = {t} code length")
    ax.axis("off")

plt.tight_layout()
plt.show()



# Magnetic Field

In [ ]:
from yt import derived_field
import yt
from PIL import Image
@derived_field(name="magnetic_field_x",units="", sampling_type="cell", display_name="Magnetic X",force_override=True)
def _mag_x(field, data):
    return data["athena_pp", "bcc1"]    
@derived_field(name="magnetic_field_y",units="", sampling_type="cell", display_name="Magnetic Y",force_override=True)
def _mag_y(field, data):
    return data["athena_pp", "bcc2"]
@derived_field(name="magnetic_field_z", units="", sampling_type="cell", display_name="Magnetic Z",force_override=True)
def _mag_z(field, data):
    return data["athena_pp", "bcc3"]
@derived_field(name="magnetic_field_total", units="", sampling_type="cell", display_name="Magnetic Field Magnitude",force_override=True)
def _mag_total(field, data):
    return np.sqrt(data["athena_pp", "bcc3"]**2 + data["athena_pp", "bcc1"]**2 + data["athena_pp", "bcc2"]**2)


import matplotlib.pyplot as plt
import numpy as np

times = [0, 50, 100]
images = []
for t in times:
    ds = yt.load(f"fm_mad/100Ms/torus.mhd_w_bcc.00{t:03d}.athdf")
    # make a slice plot of density using the disk as the data_source
    p = yt.SlicePlot(ds, "x", ('athena_pp', 'dens'))
    p.set_font_size(27)
    # p.set_cmap('dens', 'RdBu')
    # p.zoom(8)
    try:
        # p.annotate_magnetic_field()
        p.annotate_streamlines('magnetic_field_y', 'magnetic_field_z', density=1, linewidth=1, color='black')
        if t!=0:
            p.annotate_velocity()
        # p.annotate_contour(('athena_pp', 'magnetic_field_total'), ncontours=10, clim=(1e-5, 1e-2)
    except Exception:
        pass
    # save the plot and load the saved image for composing in a matplotlib figure
    saved = p.save()  # returns a list of saved file paths
    images.append(Image.open(saved[0]))

# plot all four saved images in a 2x2 layout
fig, axes = plt.subplots(1, 3, figsize=(15, 10))
for ax, img, t in zip(axes, images, times):
    ax.imshow(img)
    ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
times = [0, 50, 75, 100]
# for r in radii:
#     ds = yt.load(f"/home/yasho/athenak/visualisation/fm_mad/100Ms/torus.mhd_w_bcc.00{r:03d}.athdf")
#     prof=yt.ProfilePlot(ds, ('index','z'), ('athena_pp','velz'), weight_field=None, n_bins=100)
#     prof.show()

disk =ds.disk([0,0,0], [0,0,1], radius=(10,"code_length"),height= (1024,"code_length"))

for t in times:
    ds = yt.load(f"fm_mad/100Ms/torus.mhd_w_bcc.00{t:03d}.athdf")
    disk =ds.disk([0,0,0], [0,0,1], radius=(10,"code_length"),height= (1024,"code_length"))
    prof=yt.ProfilePlot(disk, ('index','z'), ('athena_pp','velz'), weight_field=('athena_pp','dens'), n_bins=20)
    prof.show()

In [ ]:
ad= ds.all_data()
prof = yt.create_profile(ad, ('index','cylindrical_radius'), ('athena_pp','dens'), weight_field=('athena_pp','cell_volume'), n_bins=100,logs={'cylindrical_radius': False,"dens":False})
prof.x_axis_unit = 'code_length'
prof.y_axis_unit = 'code_mass/code_length**2/code_time'
prof.plot()

# FLUX CALCULATIONS


In [ ]:
surf=ds.surface(ds.all_data(), ('index','cylindrical_radius'), 100)
surf.calculate_flux(('athena_pp','velx'), ('athena_pp','vely'), ('athena_pp','velz'),('athena_pp','dens'))

In [ ]:
surf=ds.surface(ds.all_data(), ('index','cylindrical_radius'), 101)
surf.calculate_flux(('athena_pp','velx'), ('athena_pp','vely'), ('athena_pp','velz'),('athena_pp','dens'))

In [ ]:
#find the flux across the surface for different radii and plot it against radius
radii = range(10,200,10)
fluxes = []
for r in radii:
    surf=ds.surface(ds.all_data(), ('index','cylindrical_radius'), r)
    flux = surf.calculate_flux(('athena_pp','velx'), ('athena_pp','vely'), ('athena_pp','velz'),('athena_pp','dens'))
    fluxes.append(flux)
import matplotlib.pyplot as plt
plt.plot(radii, fluxes)
plt.xlabel("Radius (code_length)")
plt.ylabel("Mass Flux (code_mass/code_time)")


In [ ]:
#find the flux across the surface for different radii and plot it against radius
radii = range(200,1000,10)
fluxes = []
for r in radii:
    surf=ds.surface(ds.all_data(), ('index','cylindrical_radius'), r)
    flux = surf.calculate_flux(('athena_pp','velx'), ('athena_pp','vely'), ('athena_pp','velz'),('athena_pp','dens'))
    fluxes.append(flux)
import matplotlib.pyplot as plt
plt.plot(radii, fluxes)
plt.xlabel("Radius (code_length)")
plt.ylabel("Mass Flux (code_mass/code_time)")

In [ ]:
disk =ds.disk([0,0,0], [0,0,1], radius=(20,"code_length"),height= (1024,"code_length"), fields=[('athena_pp','dens'), ('athena_pp','velx'), ('athena_pp','vely'), ('athena_pp','velz')])
surface = ds.surface(disk, ('index','cylindrical_z'), 10)
print(surface)
flux=surface.calculate_flux(('athena_pp','velx'), ('athena_pp','vely'), ('athena_pp','velz'),('athena_pp','dens'))



In [ ]:
print(flux)

In [ ]:
disk =ds.disk([0,0,0], [0,0,1], radius=(10,"code_length"),height= (1024,"code_length"))
for height in range(50,1000,50):
    surface = ds.surface(disk, ('index','z'), height)
    flux = surface.calculate_flux(('athena_pp','velx'), ('athena_pp','vely'), ('athena_pp','velz'),('athena_pp','dens'))
    print(f"Height: {height} Flux: {flux}")

In [ ]:
disk =ds.disk([0,0,0], [0,0,1], radius=(10,"code_length"),height= (1024,"code_length"))
prof=disk.profile(('index','z'),('athena_pp','velz'), weight_field=None, n_bins=100,logs={'z': True,"velz":True})
prof.plot()

In [ ]:
plot=yt.ProfilePlot(disk, ('index','z'), ('athena_pp','dens'), weight_field=None, n_bins=100)
plot.show()

In [ ]:
plot=yt.ProfilePlot(disk, ('index','z'), ('athena_pp','dens'), weight_field=None, n_bins=100)
plot.show()

In [ ]:
prof=ds.all_data().profile(('index','z'),('athena_pp','dens'), weight_field=None, n_bins=100,logs={'cylindrical_radius': False,"dens":False})
prof.plot()

In [ ]:
import glob
import yt

#plot the evolution of the density profile with time over all the slices in the directory
import matplotlib.pyplot as plt

# Get all files matching the pattern in the directory
files = sorted(glob.glob("/home/yasho/athenak/bin/torus.mhd_w_bcc.*.athdf"))

plt.figure(figsize=(10,6))
for f in files:
    ds = yt.load(f)
    disk=  ds.disk([0,0,0], [0,0,1], radius= (10,"code_length"), height=(1000,"code_length"))
    prof = disk.profile(('index','z'),('athena_pp','dens'), weight_field=None, n_bins=100, logs={'z': False, "dens": True})
    plt.plot(prof.x, prof['athena_pp','dens'], label=f.split('.')[-2])

plt.xlabel("z (code_length)")
plt.ylabel("Density (code_mass/code_length**3)")
plt.legend()
plt.title("Evolution of Density Profile Over Time")
plt.yscale('log')
plt.tight_layout
plt.show()


In [ ]:
import glob
import yt

#plot the evolution of the density profile with time over all the slices in the directory
import matplotlib.pyplot as plt

# Get all files matching the pattern in the directory
files = sorted(glob.glob("/home/yasho/athenak/bin/torus.mhd_w_bcc.*.athdf"))

plt.figure(figsize=(10,6))
for f in files:
    ds = yt.load(f)
    disk=  ds.disk([0,0,0], [0,0,1], radius= (1000,"code_length"), height=(200,"code_length"))
    prof = disk.profile(('index','cylindrical_radius'),('athena_pp','dens'), weight_field=None, n_bins=100, logs={'z': False, "dens": True})
    plt.plot(prof.x, prof['athena_pp','dens'], label=f.split('.')[-2])

plt.xlabel("z (code_length)")
plt.ylabel("Density (code_mass/code_length**3)")
plt.legend()
plt.title("Evolution of Density Profile Over Time")
plt.yscale('log')
plt.tight_layout